[View in Colaboratory](https://colab.research.google.com/github/hyerim1048/100_pytorch/blob/master/_1_movies_rating.ipynb)

In [2]:
!pip install pandas

In [3]:
!git clone https://github.com/e9t/nsmc.git

Cloning into 'nsmc'...
remote: Counting objects: 14763, done.
remote: Total 14763 (delta 0), reused 0 (delta 0), pack-reused 14763
Receiving objects: 100% (14763/14763), 56.19 MiB | 20.47 MiB/s, done.
Resolving deltas: 100% (1749/1749), done.
Checking out files: 100% (14737/14737), done.


In [4]:
!ls ./nsmc

code		  ratings_train.txt  raw	synopses.json
ratings_test.txt  ratings.txt	     README.md


In [0]:
!apt-get update

!apt-get install g++ openjdk-8-jdk python-dev python3-dev

!pip3 install JPype1-py3

!pip3 install konlpy

!JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"

In [0]:
import jpype

In [0]:
def read_data(filename):
    with open(filename, 'r',encoding='utf-8') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:]   # header 제외
    return data
train_data = read_data('./nsmc/ratings_train.txt')
test_data = read_data('./nsmc/ratings_test.txt')

In [7]:
print(len(train_data))      
print(len(train_data[0]))   
print(len(test_data))       
print(len(test_data[0])) 

150000
3
50000
3


In [8]:
train_data[0:10]

[['9976970', '아 더빙.. 진짜 짜증나네요 목소리', '0'],
 ['3819312', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '1'],
 ['10265843', '너무재밓었다그래서보는것을추천한다', '0'],
 ['9045019', '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', '0'],
 ['6483659',
  '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다',
  '1'],
 ['5403919', '막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.', '0'],
 ['7797314', '원작의 긴장감을 제대로 살려내지못했다.', '0'],
 ['9443947',
  '별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네',
  '0'],
 ['7156791', '액션이 없는데도 재미 있는 몇안되는 영화', '1'],
 ['5912145', '왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?', '1']]

In [9]:
from konlpy.tag import Twitter
pos_tagger = Twitter() #형태소 분석기
def tokenize(doc):
    # 형태소 분석후 [영화, Noun]을 "영화/Noun" 형태로 바꿔주는 function
    # norm, stem은 optional
    return ['/'.join(t) for t in pos_tagger.pos(doc, norm=True, stem=True)]

#모든 train data와 test data 에 적용
train_docs = [(tokenize(row[1]), row[2]) for row in train_data]
test_docs = [(tokenize(row[1]), row[2]) for row in test_data]

tokens = [t for d in train_docs for t in d[0]]
print(len((tokens))) # token 총 개수
print(len(set(tokens))) # token unique 개수

2194536
48765


In [0]:
from collections import defaultdict
tokens = defaultdict(lambda: len(tokens))

In [0]:
#word counts for negative sampling
word_counts = defaultdict(int)

S = tokens["<s>"]
UNK = tokens["<unk>"]
def read_dataset(filename):
  with open(filename, "r") as f:
    for line in f:
      line = line.strip().split(" ")
      for word in line:
        word_counts[w2i[word]] += 1
      yield [w2i[x] for x in line]


In [18]:
tokens

defaultdict(<function __main__.<lambda>>,
            {10: 3, 5: 4, 9: 2, '<s>': 0, '<unk>': 1})